In [1]:
import sys
sys.path.insert(1, '..\\..\\')

import os
import data_loader
from numpy import trapz
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Tensorflow Version

In [2]:
print(tf.__version__)

2.3.1


### Global Variables

In [4]:
malware_data_dir = '../../data/'
#saved_model_path = 'saved_model/'
#opcode_to_int_path = "opcodeToInt.txt"
num_unique_opcodes = 30
max_opcode_sequence_length = 2000
embed_vector_length = 128
num_lstm_unit = 16
dropout_amt = 0.3
batch_size = 32
num_epochs = 100
test_size= 0.15       # reserve for testing
results_path = "results.txt"
#num_families_to_use = 20

shutdown = False

In [4]:
 def split_data1(train_data_raw, train_labels_raw):
    # Split into training and testing data
    train_data, test_data, train_labels, test_labels = train_test_split(train_data_raw, train_labels_raw, test_size=test_size)

    # Make divisible by batch size
    num_data_train = int(len(train_data)/batch_size) * batch_size
    num_data_test = int(len(test_data)/batch_size) * batch_size

    train_data = train_data[:num_data_train]
    train_labels = train_labels[:num_data_train]
    test_data = test_data[:num_data_test]
    test_labels = test_labels[:num_data_test]

#     print("train_data shape: {}".format(train_data.shape))
#     print("test_data shape: {}".format(test_data.shape))
#     print("train_labels shape: {}".format(train_labels.shape))
#     print("test_labels shape: {}".format(test_labels.shape))

    return train_data, test_data, train_labels, test_labels

In [8]:
def create_model1(num_families_to_use):
    model = Sequential()
    model.add(Input(batch_shape=(batch_size, max_opcode_sequence_length), name="input"))
    model.add(Embedding(input_dim=num_unique_opcodes+1,
                                  output_dim=embed_vector_length,
                                  input_length=max_opcode_sequence_length, name="embedding"))
    # LSTM 1
    model.add(Dropout(dropout_amt, name="dropout_1"))
    model.add(LSTM(num_lstm_unit, 
                   input_shape=(None, max_opcode_sequence_length),
                   name="lstm1"))
    model.add(Dense(num_families_to_use, activation='softmax', name="dense"))
    optimizer = Adam()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    #model.summary()

    return model

In [13]:
def run_model():
    num_families_to_use_list = [5,10,15,20]
    results = {}
    
    for num_families_to_use in num_families_to_use_list:
        print("{0} families....".format(num_families_to_use))
        
        with open(results_path, 'a') as file:
            file.write(str(num_families_to_use) + "\n")
            
        opcode_to_int_path = "opcodeToInt_" + str(num_families_to_use) + ".txt"
        # Get train data
        raw_train_data = data_loader.getTrainData(malware_data_dir, 
                                                  num_families_to_use, 
                                                  num_unique_opcodes, 
                                                  max_opcode_sequence_length, 
                                                  opcode_to_int_path)
        # Data preprocessing
        family_names = list(raw_train_data.keys())
        print(family_names)

        # Split opcode family data in individual lists
        train_data = list()
        for family, data in raw_train_data.items():
            train_data.append(data)

        # Pad training data to ensure uniformity
        padded_train_data = list()
        for family_opcodes in train_data:
            padded_sequence = pad_sequences(family_opcodes, 
                                            maxlen=max_opcode_sequence_length)
            padded_train_data.append(padded_sequence)

        # Concatenate all training data into 1 long list instead of multiple lists
        train_data_raw = np.concatenate(padded_train_data)

        print(len(train_data))
        
        # Make labels
        train_labels = []
        for count, data in enumerate(padded_train_data):
            labels_list = np.full(shape=(len(data)), fill_value=count)
            train_labels.append(labels_list)

        train_labels_raw = np.concatenate(train_labels)
        
        for i in range(5):
            # get train and test data
            train_data, test_data, train_labels, test_labels = split_data(train_data_raw, train_labels_raw)

            # train model
            model_train = create_model(num_families_to_use)
            early_stopping = EarlyStopping(monitor='accuracy', verbose=1, patience=2)
            history = model_train.fit(x=train_data,
                                  y=train_labels,
                                  batch_size=batch_size,
                                  callbacks = [early_stopping],
                                  epochs=num_epochs,)

            # evaluate
            model_evaluate = create_model(num_families_to_use)
            model_evaluate.set_weights(model_train.get_weights())

            scores = model_evaluate.evaluate(test_data, test_labels, verbose=0, callbacks = [early_stopping])
            accuracy = scores[1]*100
            print("{0}: {1}".format(num_families_to_use, accuracy))
            results[num_families_to_use] = accuracy

            with open(results_path, 'a') as file:
                file.write(str(accuracy) + "\n")

In [14]:
# run_model()

5 families....
Getting list of paths to training data
{'winwebsec': 6862260, 'vundo': 3492760, 'zbot': 3256944, 'hotbar': 2952000, 'renos': 2612858}
Loading training data for hotbar
Loading training data for renos
Loading training data for vundo
Loading training data for winwebsec
Loading training data for zbot
All training data loaded
['hotbar', 'renos', 'vundo', 'winwebsec', 'zbot']
5
Epoch 1/100
265/265 [==============================] - 19s 71ms/step - loss: 1.0930 - accuracy: 0.5923
Epoch 2/100
265/265 [==============================] - 19s 70ms/step - loss: 0.9104 - accuracy: 0.6544
Epoch 3/100
265/265 [==============================] - 19s 70ms/step - loss: 0.8672 - accuracy: 0.6784
Epoch 4/100
265/265 [==============================] - 19s 70ms/step - loss: 0.7538 - accuracy: 0.7376
Epoch 5/100
265/265 [==============================] - 19s 70ms/step - loss: 0.7654 - accuracy: 0.7278
Epoch 6/100
265/265 [==============================] - 18s 68ms/step - loss: 0.7587 - accuracy:

571/571 [==============================] - 37s 65ms/step - loss: 1.5004 - accuracy: 0.5421
Epoch 9/100
571/571 [==============================] - 37s 65ms/step - loss: 1.4295 - accuracy: 0.5572
Epoch 10/100
571/571 [==============================] - 37s 65ms/step - loss: 1.5312 - accuracy: 0.5402
Epoch 11/100
571/571 [==============================] - 37s 65ms/step - loss: 1.4124 - accuracy: 0.5589
Epoch 12/100
571/571 [==============================] - 37s 65ms/step - loss: 1.4852 - accuracy: 0.5368
Epoch 13/100
571/571 [==============================] - 37s 65ms/step - loss: 1.3938 - accuracy: 0.5624
Epoch 14/100
571/571 [==============================] - 37s 65ms/step - loss: 1.3748 - accuracy: 0.5695
Epoch 15/100
571/571 [==============================] - 37s 65ms/step - loss: 1.4458 - accuracy: 0.5516
Epoch 16/100
571/571 [==============================] - 37s 65ms/step - loss: 1.3349 - accuracy: 0.5844
Epoch 17/100
571/571 [==============================] - 37s 65ms/step - loss: 

KeyboardInterrupt: 

### Load the Data

In [5]:
raw_train_data = data_loader.getTrainData(malware_data_dir, 
                                          num_families_to_use, 
                                          num_unique_opcodes, 
                                          max_opcode_sequence_length, 
                                          opcode_to_int_path)

Getting list of paths to training data
{'winwebsec': 6862260, 'vundo': 3492760, 'zbot': 3256944, 'hotbar': 2952000, 'renos': 2612858, 'onlinegames': 2554166, 'obfuscator': 2502965, 'bho': 2315982, 'alureon': 2287866, 'zeroaccess': 2238000, 'delfinject': 2167855, 'startpage': 2164599, 'adload': 2088000, 'fakerean': 2082110, 'cycbot': 2058000, 'vobfus': 1848000, 'lolyda': 1830000, 'ceeinject': 1725371, 'agent': 1625496, 'rbot': 1623452}
Finding out how many opcodes to use per family...
1623452
Generating opcode to int mapping...
File saved, done.
Loading training data for hotbar
Loading training data for renos
Loading training data for vundo
Loading training data for winwebsec
Loading training data for zbot
Loading training data for alureon
Loading training data for bho
Loading training data for obfuscator
Loading training data for onlinegames
Loading training data for zeroaccess
Loading training data for adload
Loading training data for cycbot
Loading training data for delfinject
Loadin

### Data preprocessing

In [6]:
family_names = list(raw_train_data.keys())
print(family_names)

# Split opcode family data in individual lists
train_data = list()
for family, data in raw_train_data.items():
    train_data.append(data)
    
# Pad training data to ensure uniformity
padded_train_data = list()
for family_opcodes in train_data:
    padded_sequence = pad_sequences(family_opcodes, 
                                    maxlen=max_opcode_sequence_length)
    padded_train_data.append(padded_sequence)
    
# Concatenate all training data into 1 long list instead of multiple lists
train_data_raw = np.concatenate(padded_train_data)

print(len(train_data))

['hotbar', 'renos', 'vundo', 'winwebsec', 'zbot', 'alureon', 'bho', 'obfuscator', 'onlinegames', 'zeroaccess', 'adload', 'cycbot', 'delfinject', 'fakerean', 'startpage', 'agent', 'ceeinject', 'lolyda', 'rbot', 'vobfus']
20


### Make the labels

In [7]:
train_labels = []

for count, data in enumerate(padded_train_data):
    labels_list = np.full(shape=(len(data)), fill_value=count)
    train_labels.append(labels_list)

train_labels_raw = np.concatenate(train_labels)

### Split into training and testing sets

In [8]:
def split_data(train_data_raw, train_labels_raw):
    # Split into training and testing data
    train_data, test_data, train_labels, test_labels = train_test_split(train_data_raw, train_labels_raw, test_size=test_size)

    # Make divisible by batch size
    num_data_train = int(len(train_data)/batch_size) * batch_size
    num_data_test = int(len(test_data)/batch_size) * batch_size

    train_data = train_data[:num_data_train]
    train_labels = train_labels[:num_data_train]
    test_data = test_data[:num_data_test]
    test_labels = test_labels[:num_data_test]
    
#     print("train_data shape: {}".format(train_data.shape))
#     print("test_data shape: {}".format(test_data.shape))
#     print("train_labels shape: {}".format(train_labels.shape))
#     print("test_labels shape: {}".format(test_labels.shape))
    
    return train_data, test_data, train_labels, test_labels

### Make the Model

In [9]:
def create_model():
    model = Sequential()
    model.add(Input(batch_shape=(batch_size, max_opcode_sequence_length), name="input"))
    model.add(Embedding(input_dim=num_unique_opcodes+1,
                                  output_dim=embed_vector_length,
                                  input_length=max_opcode_sequence_length, name="embedding"))
    # LSTM 1
    model.add(Dropout(dropout_amt, name="dropout_1"))
    model.add(LSTM(num_lstm_unit, 
                   input_shape=(None, max_opcode_sequence_length),
                   name="lstm1"))
#     # LSTM 2
#     model.add(Dropout(dropout_amt, name="dropout_2"))
#     model.add(LSTM(num_lstm_unit*2, 
#                    return_sequences=True,
#                    name="lstm2"))
#     # LSTM 3
#     model.add(Dropout(dropout_amt, name="dropout_3"))
#     model.add(LSTM(num_lstm_unit, 
#                                    name="lstm3"))
    model.add(Dense(num_families_to_use, activation='softmax', name="dense"))
    optimizer = Adam()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    #model.summary()
    
    return model

In [10]:
results = []

for i in range(5):
    # get train and test data
    train_data, test_data, train_labels, test_labels = split_data(train_data_raw, train_labels_raw)
    
    # train model
    model_train = create_model()
    early_stopping = EarlyStopping(monitor='loss', verbose=1, patience=2)
    history = model_train.fit(x=train_data,
                          y=train_labels,
                          batch_size=batch_size,
                          callbacks = [early_stopping],
                          epochs=num_epochs,)
    
    # evaluate
    model_evaluate = create_model()
    model_evaluate.set_weights(model_train.get_weights())

    scores = model_evaluate.evaluate(test_data, test_labels, verbose=0, callbacks = [early_stopping])
    accuracy = scores[1]*100
    print(accuracy)
    results.append(accuracy)

Epoch 1/100
687/687 [==============================] - 48s 69ms/step - loss: 2.4105 - accuracy: 0.2950
Epoch 2/100
687/687 [==============================] - 46s 68ms/step - loss: 2.0392 - accuracy: 0.4087
Epoch 3/100
687/687 [==============================] - 47s 68ms/step - loss: 1.9072 - accuracy: 0.4486
Epoch 4/100
687/687 [==============================] - 46s 68ms/step - loss: 1.8135 - accuracy: 0.4735
Epoch 5/100
687/687 [==============================] - 46s 67ms/step - loss: 1.7784 - accuracy: 0.4808
Epoch 6/100
687/687 [==============================] - 47s 68ms/step - loss: 1.7237 - accuracy: 0.4963
Epoch 7/100
687/687 [==============================] - 46s 67ms/step - loss: 1.9222 - accuracy: 0.4390
Epoch 8/100
687/687 [==============================] - 45s 66ms/step - loss: 1.8078 - accuracy: 0.4730
Epoch 00008: early stopping
49.793389439582825
Epoch 1/100
687/687 [==============================] - 47s 69ms/step - loss: 2.3920 - accuracy: 0.3156
Epoch 2/100
687/687 [=====

In [11]:
for x in results:
    print(x)

# model_train.save_weights(saved_model_path) 

49.793389439582825
48.063015937805176
45.17045319080353
55.24277091026306
51.80785059928894


### Use History to plot and accuracy throughout training

In [ ]:
history.history

### Evaluate Model

In [ ]:
model_evaluate = create_model()
model_evaluate.set_weights(model_train.get_weights())

scores = model_evaluate.evaluate(test_data, test_labels, verbose=0, callbacks = [early_stopping])
accuracy = scores[1]*100
print("Accuracy: %0.2f%%" % (scores[1]*100))

### Load Model From Save and Evaluate

In [ ]:
model = create_model()
model.load_weights(saved_model_path)



In [ ]:
scores = model.evaluate(test_set, test_labels, verbose=0)
accuracy = scores[1]*100
print("Accuracy: %0.2f%%" % (scores[1]*100))

## Make ROC Curve

### Make scatter plot

In [ ]:
# separate winwebsec and zbot test data
winwebsec_test_data = []
zbot_test_data = []

for i in range(len(test_labels)):
    if test_labels[i] == 0:
        winwebsec_test_data.append(test_set[i])
    else:
        zbot_test_data.append(test_set[i])
        
winwebsec_test_data = np.asarray(winwebsec_test_data[:192])
zbot_test_data = np.asarray(zbot_test_data[:128])

In [ ]:
print(winwebsec_test_data.shape)
print(zbot_test_data.shape)


winwebsecY = model.predict(winwebsec_test_data)
winwebsecX = [i+1 for i in range(len(winwebsec_test_data))]

zbotY = model.predict(zbot_test_data)
zbotX = [i+1 for i in range(len(zbot_test_data))]

In [ ]:
plt.figure(100)
f = plt.scatter(winwebsecX, winwebsecY, marker='o',
                c='darkblue', s=30, label="winwebsec")
plt.scatter(zbotX, zbotY, marker='o', c='red', s=30, label="zbot")
plt.title("Winwebsec vs. Zbot LSTM Prediction Scatter Plot",
          fontsize=18, wrap=True)
f.axes.get_xaxis().set_visible(False)
plt.ylabel("Prediction", fontsize=15)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize=12)

### Make ROC Curve

In [ ]:
def sortByFirstItem(item):
    return item[0]

In [ ]:
winwebsecROC = [(data, "winwebsec") for data in winwebsecY]
zbotROC = [(data, "zbot") for data in zbotY]

zbotROC.sort(key=sortByFirstItem)
winwebsecROC.sort(key=sortByFirstItem)

dataROC = zbotROC + winwebsecROC
dataROC.sort(key=sortByFirstItem, reverse=True)

In [ ]:
def calculate_TPR_FPR(thresholdLine, dataROC):
    TP = 0
    FN = 0
    TN = 0
    FP = 0

    for data in dataROC:
        yVal = data[0]
        family = data[1]

        if family == "winwebsec":
            if yVal < thresholdLine:
                TP += 1
            else:
                FN += 1
        elif family == "zbot":
            if yVal > thresholdLine:
                TN += 1
            else:
                FP += 1
    TPR = TP/(TP+FN)
    FPR = 1 - (TN/(TN+FP))

    return TPR, FPR

In [ ]:
def calculateAUC(rocData):
    sum = 0

    # initialization
    prevX = -1
    prevY = -1

    for points in rocData:
        curX = points[0]
        curY = points[1]

        # Skip for first point
        if prevX != -1 and prevY != -1:
            # check if rectangle
            if prevY == curY:
                sum += abs(curX - prevX) * prevY
            # check if trapezoid
            else:
                sum += (curY + prevY) * abs(curX - prevX) * 0.5

        prevX = curX
        prevY = curY

    return sum

In [ ]:
rocX = list()  # used to plot
rocY = list()  # used to plot
rocData = list()    # used to calculate AUC

for entry in dataROC:
    thresholdLine = entry[0]
    TPR, FPR = calculate_TPR_FPR(thresholdLine, dataROC)

    rocX.append(FPR)
    rocY.append(TPR)
    rocData.append([FPR, TPR])

rocData.sort(key=lambda item: (item[0], item[1]), reverse=True)

AUC = round(calculateAUC(rocData), 3)

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

plt.figure(200)
plt.plot(rocX, rocY, marker=".", markersize=8)
plt.title("Winwebsec vs. Zbot LSTM Log Probability ROC", fontsize=18)
plt.xlabel("FPR", fontsize=15)
plt.ylabel("TPR", fontsize=15)
plt.grid()
plt.text(x=0.75, y=0, s="AUC: {0}".format(AUC), fontsize=14, bbox=props)

# show plots
plt.show()

In [ ]:
if shutdown:
    os.system('shutdown -s')